In [ ]:
def memeContinent(str_air1, str_air2):
    '''
    fonction qui retourne vrai si les deux sont sur le meme continent 
    et faux sinon
    '''
    c1 = dict_aeroports[str_air1]['Country']
    c2 = dict_aeroports[str_air2]['Country']
    
    return c1 == c2

In [ ]:
def nettoyageList(str_list, str_airStart, str_airEnd):
    '''
    fonction qui supprimer les occurences dans le liste qui ne sont pas sur le 
    meme continent que les airport de depart et d'arrive . 
    '''
    
    if memeContinent(str_airStart, str_airEnd):
        #print('Les airports sont sur le même continent ')
        compteur = 0 ; 
        for element in str_list:
            if not memeContinent(str_airStart, element):
                str_list.remove(element)
                compteur = compteur + 1 
    
    #print('On a enleve {c} airports du list'.format(c = compteur))
    return str_list  
    

In [ ]:
def creationEdgesConnection(graphe : nx.Graph, list_reference : list, code1 : str, code2 : str, list_connections_nettoye):
    '''
    fonction qui genere des edges pour notre graphe avec les weights. On appele seulement cette
    focntion si on a le list_connections (qui sont les airports entre code1 et code2)
    retour : list_ref (qui a les index qui corresponds au nodes du graph)
    '''
        
    # pour gerer les correspondances 
    if list_reference.count(code1) == 0: 
        list_reference.append(code1)
        graphe.add_node(list_reference.index(code1))   
    if list_reference.count(code2) == 0: 
        list_reference.append(code2)
        graphe.add_node(list_reference.index(code1))
    
    for i in list_connections_nettoye :
        if list_reference.count(i) == 0 : 
            list_reference.append(i)   
            graphe.add_node(list_reference.index(i))  
    
    index1 = list_reference.index(code1) 
    index2 = list_reference.index(code2)
    
    list_edges = list(map(lambda x :
        (index1, list_reference.index(x), int(DistGrandCercleICAO(code1, x))),
        list_connections_nettoye))
    
    list_edges.extend(list(map(lambda y : 
        (list_reference.index(y), index2, int(DistGrandCercleICAO(y, code2))),
        list_connections_nettoye)))
    
    graphe.add_weighted_edges_from(list_edges)
    return list_reference

In [ ]:
def creationEdgesAucunConnection(graphe : nx.Graph, list_reference : list, code1 : str, code2 : str):
    print(code1, code2)
    posCoteStart = df_vols[df_vols['Depart'] == code1]['Depart'].to_list()
    posCoteEnd = df_vols[df_vols['Arrivee'] == code2]['Depart'].to_list()
    
    if posCoteStart == []: print('Dans notre database, il y a pas des vols qui part de {} .'.format(code1)) ; return False
    if posCoteEnd == []: print('Dans notre database, il y a pas des vols qui arrive sur {} .'.format(code2)); return False
    
    # des le moment que il trouve une chemin pour reliee les deux airports il le fait 
    for i_code in posCoteStart:
        for j_code in posCoteEnd: 
            # verifie si on a une connection 
            tmp = correspondanceAirport(i_code, j_code)
            # si on a une conenction 
            if len(tmp) > 0 :
                # on ajoute les premiere elements 
                list_reference.extend([code1, code2, i_code, j_code])
                edgeStart = (list_reference.index(code1), list_reference.index(i_code), int(DistGrandCercleICAO(code1, i_code)))
                edgeEnd = (list_reference.index(j_code), list_reference.index(code2), int(DistGrandCercleICAO(j_code, code2)))
                
                graphe.add_weighted_edges_from([edgeStart, edgeEnd])
                return creationEdgesConnection(graphe, list_reference, i_code, j_code, tmp)
                
            
    print('Aucun possiblite pour relie des airports avec trois correspondances (qui est notre max predifini')

In [ ]:
def creationEdges(graphe : nx.Graph, list_reference : list, code1 : str, code2 : str):
    '''
        fonction qui crée à partir d'une liste d'aéroports un list des edges d'une graphe (avec le weight)
        et les ajouts dans le graph donne en parametre 
        retour : list des edges ajoute a la graphe [(start , end, weight), (start, end ,weight ) .. ]
    '''
    # dans le listReference les airport recois un index qui corresponds au identifiant de graphe 
    list_connections = correspondanceAirport(code1, code2)
    list_connections_nettoye = list_connections
    
    if(len(list_connections) > 0 ):
      return creationEdgesConnection(graphe, list_reference, code1, code2, list_connections_nettoye)
    else :
        return print("let op functie"); creationEdgesAucunConnection(graphe, list_reference, code1, code2)

In [ ]:
def dessinerGraphe(graphe, weighted : bool = False):
    fig = plt.figure(); 
    fig.set_size_inches(15, 15)
    fig.patch.set_facecolor('white')
 
    pos = nx.fruchterman_reingold_layout(graphe)
    nx.draw_networkx_nodes(graphe, pos, node_size = 200)
    nx.draw_networkx_edges(graphe,pos, edgelist= graphe.edges(), edge_color='black')
    if graphe.number_of_nodes() < 2000:
        nx.draw_networkx_labels(graphe, pos)
    plt.autoscale()
   
    if(weighted): nx.draw_networkx_edge_labels(graphe, pos) # afficher le weight 

    plt.show()

In [ ]:
def nombreDeCorrespondances(g : nx.Graph, codeStart, codeFin):
    listRetour = creationEdges(g, [], codeStart, codeFin)
    ## was not  
    if listRetour == [] : 
        
        print('check in nombreDeCorrespondance Edges : ', len(listRetour), g.number_of_edges())
        
        print('check in nombreDeCorrespondance: ', len(listRetour), g.number_of_nodes())
            
        if g.has_edge(listRetour.index(codeStart), listRetour.index(codeFin)): g.remove_edge(listRetour.index(codeStart), listRetour.index(codeFin))
        if g.has_edge(listRetour.index(codeFin), listRetour.index(codeStart)): g.remove_edge(listRetour.index(codeFin), listRetour.index(codeStart))
        
        return listRetour

    return listRetour

In [ ]:
def traitementRequest(aeroport_start : str, aeroport_fin : str, g : nx.Graph = nx.Graph()):
    '''
        fonction qui fait le creation de graphe et qui transofrme les nom de airports en node de graphes
        retour : graphe, list_ref (qui est le lien entre le nombr de nodes et le code de airports)
    '''
    aeroport_codeS = filtreDictAeroports('Name', aeroport_start)[0]
    aeroport_codeE = filtreDictAeroports('Name', aeroport_fin)[0] 
    list_ref = [] 
    list_ref = creationEdges(g, list_ref, aeroport_codeS, aeroport_codeE)
    
    return g ,list_ref
    
def distancePlusCourte(graphe: nx.Graph, list_ref : list, aeroport_codeS, aeroport_codeE):
    '''
        fonction qui trouve le chemin de start aux fin avec le moins de distance possible 
        retour : dictionnaire en forme de {'distance' : .. , 'path' : [ path1, path2 ]}
    '''
    resultat = dijkstra(graphe, list_ref.index(aeroport_codeS), list_ref.index(aeroport_codeE))
    if(resultat == -1 ):
        print('No path (distance plus courte)')
        return -1 
    else : 
        resultat['path'] = list(map(lambda x : list_ref[x], resultat['path']))
        return resultat 

def routeAlternatives(graphe : nx.Graph, list_ref : list, int_start : int, int_end : int):
    '''
        fonction qui trouve tous les chemins de start aux fin avec le meme nombr de correspondances 
        que le route plus courte. 
        retour : dictionnaire en forme de {'nbRoute' : .. , 'path' : [ path1, path2 ]}
    '''

    r = list(nx.all_shortest_paths(graphe,source=int_start,target=int_end))  
    
    tmp = []; 
    for i in r: 
        tmp.append(list(map(lambda x : list_ref[x], i)))
        
    return {'nbRoute' : len(r), 'path' : tmp} 


In [ ]:
## marche seulement pour les petit relations 
def routeSelonNombreCorrespondances(graphe : nx.Graph, nombre : int, start, end):
    '''
        fonction qui trouve tous les chemins de start aux fin avec 'nombre' correspondances
        retour : dictionnaire en forme de {'nbRoute' : .. , 'path' : [ path1, path2 ]}
    '''
    r2 = list(nx.all_simple_paths(graphe,source=start,target=end))    
    r3 = list(filter(lambda x : len(x) == nombre, r2))
    
    return {'nbRoute' : len(r3), 'path': r3}

def routeSelonNombreCorrespondanceLessThen(graphe : nx.Graph, nombre : int, start, end):
    '''
        fonction qui retourne tous les routes/correspondances avec un nombre de correspondances <= nombre 
        retour : un dictionnaire { nombre : {'nbRoute' : .. , 'path' : [ path1, path2 ]}, nombre2 : {..}}
    '''
    r1 = list(nx.all_simple_paths(graphe,source=start,target=end))    
    r2 = list(filter(lambda x : len(x) <= nombre, r1))
    
    return {'nbRoute' : len(r2), 'path': r2}
